# 흐름 요약

## BAM 태그 미리보기 → Cell Ranger가 바코드/UMI/유전자 태그를 어떻게 붙였는지

(바코드, 유전자) 한 쌍 선택 →

BAM에서 UB 고유개수 세기 ↔ matrix.mtx의 값 대조 (동일해야 함)

CR/UR ↔ CB/UB 비교 → 화이트리스트/오류교정이 실제로 어떻게 적용됐는지 확인

In [1]:
import os, gzip, pandas as pd
from collections import defaultdict
import numpy as np
import scipy.io
import pysam

RUN = os.path.expanduser("/Users/gimjuyeong/Desktop/single-cell/Data/PBMC 1K")   #결과 파일 경로 지정 (각 파일 형태가 달라서)
OUT = os.path.join(RUN, "outs")      
BAM = os.path.join(OUT, "possorted_genome_bam.bam")  # read alignmnet result
BAR = os.path.join(OUT, "filtered_feature_bc_matrix", "barcodes.tsv.gz") #barcode list (singlet - positive result)
FEA = os.path.join(OUT, "filtered_feature_bc_matrix", "features.tsv.gz") #feature information (gene ID, gene name etc..)
MTX = os.path.join(OUT, "filtered_feature_bc_matrix", "matrix.mtx.gz")  #sparse matrix

for p in [BAM, BAR, FEA, MTX]:
    assert os.path.exists(p), f"파일이 없습니다: {p}"

bam = pysam.AlignmentFile(BAM, "rb")

# barcodes, features 로드
barcodes = [line.strip() for line in gzip.open(BAR, "rt")]
feat_rows = [line.strip().split("\t") for line in gzip.open(FEA, "rt")]
# features.tsv는 [gene_id, gene_name, feature_type] 3열
feat_df = pd.DataFrame(feat_rows, columns=["gene_id","gene_name","feature_type"])

# matrix.mtx (CSR) 로드
mtx = scipy.io.mmread(MTX).tocsr().T  # (barcodes x features)
print(mtx.shape, len(barcodes), len(feat_df))

[W::hts_idx_load3] The index file is older than the data file: /Users/gimjuyeong/Desktop/single-cell/Data/PBMC 1K/outs/possorted_genome_bam.bam.bai


(1221, 38606) 1221 38606


In [2]:
# 몇 개 리드만 태그 보기
cnt = 0
for read in bam.fetch(until_eof=True):
    tags = dict(read.get_tags())
    has = set(tags.keys()) & {"CB","CR","UB","UR","GX","GN"}
    if has:
        print("READ:", read.query_name)
        for k in sorted(has):
            print(f"  {k}:", tags[k])
        cnt += 1
    if cnt >= 3:
        break


READ: A00228:279:HFWFVDMXX:2:1385:2085:18975
  CR: TCTGCATGTAACTTAA
  UB: TACCACAACCAG
  UR: TACCACAACCAG
READ: A00228:279:HFWFVDMXX:1:2388:30499:30827
  CB: CACTGAAGTCTTTCTA-1
  CR: CACTGAAGTCTTTCTA
  GN: DDX11L2
  GX: ENSG00000290825
  UB: GCCTTACAGGTC
  UR: GCCTTACAGGTC
READ: A00228:279:HFWFVDMXX:1:2415:3640:8015
  CB: CACTGAAGTCTTTCTA-1
  CR: CACTGAAGTCTTTCTA
  GN: DDX11L2
  GX: ENSG00000290825
  UB: GCCTTACAGGTC
  UR: GCCTTACAGGTC


In [3]:
# load matrix
raw = scipy.io.mmread(MTX)   # sparse matrix (COO usually)
print("raw matrix type:", type(raw), "shape:", raw.shape)
mtx = raw.tocsr()
print("as CSR shape:", mtx.shape)

# quick sanity prints
print("len(barcodes) =", len(barcodes))
print("len(features) =", len(feat_df))
print("mtx.shape =", mtx.shape)

# If neither dim matches barcodes, error out with diagnostic info
if mtx.shape[0] == len(barcodes):
    print("=> rows correspond to barcodes (no transpose needed).")
elif mtx.shape[1] == len(barcodes):
    print("=> columns correspond to barcodes; transposing matrix to make rows=barcodes.")
    mtx = mtx.T.tocsr()
    print("new mtx.shape =", mtx.shape)
else:
    # 상세진단: 출력해서 왜 안맞는지 확인
    raise RuntimeError(f"barcode 길이({len(barcodes)})가 mtx 차원과 맞지 않습니다. mtx.shape={mtx.shape}. "
                       "filtered vs raw matrix 경로가 맞는지 확인하세요.")

# 이제 row-wise nonzero 검사 (rows == barcodes)
row_nnz = np.diff(mtx.indptr)  # length == n_rows
print("n_rows (barcodes) =", mtx.shape[0], "n_cols (features) =", mtx.shape[1])
print("row_nnz: min, median, max =", row_nnz.min(), np.median(row_nnz), row_nnz.max())

# 안전하게 rid 선택: nonzero가 있는 행들만 대상으로
nonzero_rows = np.where(row_nnz > 0)[0]
if len(nonzero_rows) == 0:
    raise RuntimeError("모든 바코드 행이 0입니다. mtx / barcodes 파일이 올바른지 확인하세요.")
rid = int(nonzero_rows[np.argmax(row_nnz[nonzero_rows])])  # nonzero 중에서 가장 많은 것을 골라
print("선택한 rid (0-based):", rid, "row_nnz[rid] =", row_nnz[rid])

# 범위 재확인 (안전장치)
if rid < 0 or rid >= len(barcodes):
    raise IndexError(f"계산된 rid={rid} 가 barcodes 인덱스 범위(0..{len(barcodes)-1}) 밖에 있습니다.")

BC = barcodes[rid]
print("선택 바코드:", BC)

# 그 바코드에서 가장 카운트가 높은 유전자 찾기 (행이 barcode, 열이 gene)
row_start, row_end = mtx.indptr[rid], mtx.indptr[rid+1]
cols = mtx.indices[row_start:row_end]
vals = mtx.data[row_start:row_end]
if len(vals)==0:
    raise RuntimeError("선택한 바코드 행에 nonzero 값이 없음.")
cid = int(cols[np.argmax(vals)])
VAL = int(vals.max())
GID = feat_df.iloc[cid, 0]
GNAME = feat_df.iloc[cid, 1]
print("선택 유전자:", GID, GNAME, "열인덱스:", cid, "matrix값:", VAL)


raw matrix type: <class 'scipy.sparse._coo.coo_matrix'> shape: (38606, 1221)
as CSR shape: (38606, 1221)
len(barcodes) = 1221
len(features) = 38606
mtx.shape = (38606, 1221)
=> columns correspond to barcodes; transposing matrix to make rows=barcodes.
new mtx.shape = (1221, 38606)
n_rows (barcodes) = 1221 n_cols (features) = 38606
row_nnz: min, median, max = 15 3290.0 8132
선택한 rid (0-based): 624 row_nnz[rid] = 8132
선택 바코드: GATCACACACCCTGTT-1
선택 유전자: ENSG00000198938 MT-CO3 열인덱스: 38575 matrix값: 1211


### Cell Ranger의 demultiplexing/교정 과정이 실제로 barcode/UMI를 어떻게 바꿨는지 확인

In [4]:
diff_examples = []
for read in bam.fetch(until_eof=True):
    tags = dict(read.get_tags())   # read의 태그들을 딕셔너리로 가져옴
    cb, cr = tags.get("CB"), tags.get("CR")   # 교정된 바코드, 원시 바코드
    ub, ur = tags.get("UB"), tags.get("UR")   # 교정된 UMI, 원시 UMI
    
    # (1) cell barcode 비교
    if cb and cr and cb != cr:
        diff_examples.append(("CB!=CR", read.query_name, cr, cb))
    
    # (2) UMI 비교
    if ub and ur and ub != ur:
        diff_examples.append(("UB!=UR", read.query_name, ur, ub))
    
    # (3) 예시 5개만 모으고 종료
    if len(diff_examples) >= 5:
        break

# 보기 편하게 DataFrame으로 정리
pd.DataFrame(diff_examples, columns=["type","read_id","raw","corrected"])


,type,read_id,raw,corrected
0,CB!=CR,A00228:279:HFWFVDMXX:1:2317:2790:31093,CACTGAAGTCTTTCTA,CACTGAAGTCTTTCTA-1
1,CB!=CR,A00228:279:HFWFVDMXX:2:2166:6289:9565,CACTGAAGTCTTTCTA,CACTGAAGTCTTTCTA-1
2,CB!=CR,A00228:279:HFWFVDMXX:1:1276:28908:3991,TACCGGGTCCTCGATC,TACCGGGTCCTCGATC-1
3,CB!=CR,A00228:279:HFWFVDMXX:2:1481:8359:5447,GTGTTCCGTAGTTCCA,GTGTTCCGTAGTTCCA-1
4,CB!=CR,A00228:279:HFWFVDMXX:2:1126:28537:34350,AGGTGTTAGCAATTAG,AGGTGTTAGCAATTAG-1


In [5]:
total_reads = 0
cb_diff, ub_diff = 0, 0

for read in bam.fetch(until_eof=True):
    total_reads += 1
    tags = dict(read.get_tags())
    cb, cr = tags.get("CB"), tags.get("CR")
    ub, ur = tags.get("UB"), tags.get("UR")
    
    # cell barcode 교정 여부
    if cb and cr and cb != cr:
        cb_diff += 1
    
    # UMI 교정 여부
    if ub and ur and ub != ur:
        ub_diff += 1

print("총 리드 수:", total_reads)
print("Cell barcode 교정된 리드 수:", cb_diff, f"({cb_diff/total_reads*100:.3f}%)")
print("UMI 교정된 리드 수:", ub_diff, f"({ub_diff/total_reads*100:.3f}%)")


총 리드 수: 66601879
Cell barcode 교정된 리드 수: 64866118 (97.394%)
UMI 교정된 리드 수: 1096133 (1.646%)


### CellRanger - outs 

- Cell Ranger 결과 디렉토리 구조를 따라 파일 경로를 지정함.

- outs/ 안에 주요 결과들이 들어 있음:

- possorted_genome_bam.bam: read alignment 결과 (각 read가 genome 어디에 붙었는지)

- barcodes.tsv.gz: 유효 세포(barcode) 리스트

- features.tsv.gz: feature 정보 (유전자 ID, 유전자명, feature 종류)

- matrix.mtx.gz: 유전자 × barcode sparse matrix

In [6]:
# 상위 N개의 바코드 × 각 바코드 상위 M 유전자에 대해
N, M = 5, 5
top_rows = np.argsort(row_nnz)[-N:][::-1]

def bam_umi_count_for(BC, GID, max_scan=3_000_000):
    umis, reads = set(), 0
    i = 0
    for read in bam.fetch(until_eof=True):
        i += 1
        tags = dict(read.get_tags())
        if tags.get("CB")==BC and tags.get("GX")==GID:
            reads += 1
            ub = tags.get("UB")
            if ub: umis.add(ub)
        if i >= max_scan:
            break
    return reads, len(umis)

rows = []
for rid in top_rows:
    BC = barcodes[rid]
    rs, re = mtx.indptr[rid], mtx.indptr[rid+1]
    cols, vals = mtx.indices[rs:re], mtx.data[rs:re]
    top_cidx = cols[np.argsort(vals)[-M:][::-1]]
    for cid in top_cidx:
        GID, GNAME = feat_df.iloc[cid, 0], feat_df.iloc[cid, 1]
        VAL = int(mtx[rid, cid])
        reads_n, umis_n = bam_umi_count_for(BC, GID)  # 주의: 큰 BAM에선 느릴 수 있음
        rows.append([BC, GID, GNAME, VAL, reads_n, umis_n])

report = pd.DataFrame(rows, columns=["barcode","gene_id","gene","matrix_val(UMI)","reads_in_BAM","UMI_in_BAM"])
report


,barcode,gene_id,gene,matrix_val(UMI),reads_in_BAM,UMI_in_BAM
0,GATCACACACCCTGTT-1,ENSG00000198938,MT-CO3,1211,0,0
1,GATCACACACCCTGTT-1,ENSG00000198712,MT-CO2,1210,0,0
2,GATCACACACCCTGTT-1,ENSG00000198804,MT-CO1,1190,0,0
3,GATCACACACCCTGTT-1,ENSG00000075624,ACTB,870,0,0
4,GATCACACACCCTGTT-1,ENSG00000198899,MT-ATP6,858,0,0
5,CGAGAAGTCAAGTTGC-1,ENSG00000211592,IGKC,10915,0,0
6,CGAGAAGTCAAGTTGC-1,ENSG00000253755,IGHGP,1885,0,0
7,CGAGAAGTCAAGTTGC-1,ENSG00000198938,MT-CO3,1019,0,0
8,CGAGAAGTCAAGTTGC-1,ENSG00000198712,MT-CO2,851,0,0
9,CGAGAAGTCAAGTTGC-1,ENSG00000198804,MT-CO1,779,0,0


## Load data

### workshop - adata 적용

In [7]:
import scanpy as sc
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import scanpy as sc

sc.__version__

'1.11.2'

In [8]:
adata = sc.read_10x_h5('/home/r22100180/sample_GRCh38/outs/filtered_feature_bc_matrix.h5')
adata

FileNotFoundError: [Errno 2] Unable to synchronously open file (unable to open file: name = '/home/r22100180/sample_GRCh38/outs/filtered_feature_bc_matrix.h5', errno = 2, error message = 'No such file or directory', flags = 0, o_flags = 0)

In [ ]:
adata.var_names_make_unique()

In [ ]:
adata

## The count data matrix

In [ ]:
adata.X

In [ ]:
adata.X.data

In [ ]:
adata.X.toarray()

## Cell barcodes

In [ ]:
adata.obs

In [ ]:
type(adata.obs)

## Genes

In [ ]:
adata.var

In [ ]:
type(adata.var)

## Sparsity affects gene-gene relationships

In [ ]:
sc.pl.scatter(adata, x = 'CD4', y = 'CD8A', size = 50)

In [ ]:
from scipy.stats import gaussian_kde

x = adata[:, 'CD4'].X.toarray().flatten()
y = adata[:, 'CD8A'].X.toarray().flatten()

# Calculate the point density
xy = np.vstack([x,y])
z = gaussian_kde(xy)(xy)

In [ ]:
fig, ax = plt.subplots()
im1 = ax.scatter(x, y, c=z, s=100)
ax.set_xlabel('CD4')
ax.set_ylabel('CD8A')
fig.colorbar(im1)
plt.show()


## Calculate QC metrics

In [ ]:
sc.pp.calculate_qc_metrics(adata, inplace=True)

In [ ]:
adata

In [ ]:
adata.obs.head()

In [ ]:
adata.var.head()

## Visualize library size

In [ ]:
plt.hist(adata.obs['log1p_total_counts'], 100);
plt.xlabel('log1p library size', fontsize = 14)
plt.ylabel('Frequency', fontsize = 14)

## Visualize the number of genes in each cell

In [ ]:
plt.hist(adata.obs['log1p_n_genes_by_counts'], bins = 100);
plt.xlabel('Log of num. genes per cell', fontsize = 14)
plt.ylabel('Frequency', fontsize = 14)
#plt.title('Histogram of number of genes expressed in each cell', fontsize = 14)

In [ ]:
fig = plt.figure(figsize = (8*3, 6*1))
ax = fig.add_subplot(1, 3, 1)
ax.hist(adata.obs['log1p_total_counts'], bins = 100);
ax.set_xlabel('Log-library size', fontsize = 14)
ax.set_ylabel('Frequency', fontsize = 14)
ax.set_title('Histogram of log library size', fontsize = 14)

ax = fig.add_subplot(1, 3, 2)
ax.hist(adata.obs['log1p_n_genes_by_counts'], bins = 100);
ax.set_xlabel('Log of num. genes per cell', fontsize = 14)
ax.set_ylabel('Frequency', fontsize = 14)
ax.set_title('Histogram of number of genes expressed in each cell', fontsize = 14)

ax = fig.add_subplot(1, 3, 3)
x = adata.obs['log1p_total_counts']
y = adata.obs['log1p_n_genes_by_counts']
ax.scatter(x, y, s = 5);
ax.set_ylabel('Log of num. genes per cell', fontsize = 14)
ax.set_xlabel('Log library size', fontsize = 14)
corr_coef = np.corrcoef(x, y)[0, 1]
ax.set_title('Correlation = ' + str(round(corr_coef, 3)), fontsize = 14)

## Mitochondrial content

In [ ]:
adata.var['mt'] = adata.var_names.str.startswith('MT-')  # annotate the group of mitochondrial genes as 'mt'

In [ ]:
sc.pp.calculate_qc_metrics(adata, qc_vars=['mt'], percent_top=None, log1p=False, inplace=True)

In [ ]:
adata

### Visualize Mito-content

In [ ]:
fig = plt.figure(figsize = (8*3, 6*1))
ax = fig.add_subplot(1, 3, 1)
ax.hist(adata.obs['pct_counts_mt'], 100);
ax.set_xlabel('% MT-content', fontsize = 14)
ax.set_ylabel('Frequency', fontsize = 14)

ax = fig.add_subplot(1, 3, 2)
ax.scatter(adata.obs['log1p_total_counts'], adata.obs['pct_counts_mt']);
ax.set_xlabel('Log library size', fontsize = 14)
ax.set_ylabel('% MT-content', fontsize = 14)

ax = fig.add_subplot(1, 3, 3)
ax.scatter(adata.obs['log1p_n_genes_by_counts'], adata.obs['pct_counts_mt']);
ax.set_xlabel('Log num. genes per cell', fontsize = 14)
ax.set_ylabel('% MT-content', fontsize = 14)

In [ ]:
adata_clean = adata[adata.obs['pct_counts_mt'] < 25, :]

In [ ]:
adata_clean

In [ ]:
# just double check: there must be no cell with MT > 25
plt.hist(adata_clean.obs['pct_counts_mt'], 100);
plt.xlabel('% MT-content', fontsize = 14)
plt.ylabel('Frequency', fontsize = 14)

## Plot number of genes expressed in each cell

In [ ]:
plt.hist(adata_clean.var['n_cells_by_counts'], bins = 100);
plt.xlabel('Number of cells a gene is expressed in', fontsize = 14)
plt.ylabel('Frequency', fontsize = 14)
plt.title('Histogram of number of cells each gene is expressed in', fontsize = 14)

In [ ]:
fig = plt.figure(figsize = (8*3, 6*1))
ax = fig.add_subplot(1, 3, 1)
ax.hist(adata_clean.var['n_cells_by_counts'], bins = 100);
ax.set_xlabel('Number of cells a gene is expressed in', fontsize = 14)
ax.set_ylabel('Frequency', fontsize = 14)
ax.set_title('Histogram of number of cells each gene is expressed in', fontsize = 14)

ax = fig.add_subplot(1, 3, 2)
ax.hist(np.log(adata_clean.var['n_cells_by_counts'] + 1), bins = 100);
ax.set_xlabel('Log - Number of cells a gene is expressed in', fontsize = 14)
ax.set_ylabel('Frequency', fontsize = 14)
ax.set_title('Histogram of log of number of cells each gene is expressed in', fontsize = 14)

ax = fig.add_subplot(1, 3, 3)
ax.hist(np.log(adata_clean.var['n_cells_by_counts'] + 1), bins = 100);
ax.set_xlabel('Log - Number of cells a gene is expressed in', fontsize = 14)
ax.set_ylabel('Frequency, ylim trimmed', fontsize = 14)
ax.set_title('Histogram of log of number of cells each gene is expressed in', fontsize = 14)
ax.set_ylim([0, 1000])

### Remove lowly expressed genes

In [ ]:
sc.pp.filter_genes(adata_clean, min_cells = np.exp(4))

In [ ]:
adata_clean

## Ribosomal RNA content

In [ ]:
adata_clean.var_names[adata_clean.var_names.str.startswith(('RPS', 'RPL'))]

In [ ]:
adata_clean = adata_clean[:, ~adata_clean.var_names.str.startswith(('RPS', 'RPL'))]

In [ ]:
adata_clean